In [2]:
#!/usr/bin/env python

"""MXGSTGF_Observation_test_Data_read_into_db.py: read raw data in database"""


__author__ 		= "Gareth Murphy"
__copyright__ 	= "Copyright 2016, ASDC, DTU Space"
__credits__ 	= ["spacepy, bitstring, textwrap, numpy"]
__license__ 	= "XXX"
__version__ 	= "0.0.1"
__maintainer__ 	= "Gareth Murphy"
__email__ 		= "gmurphy@space.dtu.dk"
__status__ 		= "Working version, finds and reads data correctly "

import os, sys


home="/Users/garmu/Dropbox/"
proj_path = home+"/asdc/web/python_template_site"

proj_path = home+"/myenv/mysite/"
# This is so Django knows where to find stuff.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mysite.settings")
sys.path.append(proj_path)

# This is so my local_settings.py gets loaded.
os.chdir(proj_path)

# This is so models get loaded.
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


from asim.models import * 
    

In [3]:


def hex2int(charstr):
    return int(charstr,16)


def MXGSTGFObservation_to_db(fname):

    path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/raw/'


    #fname='PFM_20160513_0912_raw.txt'
    i=0

    import re


    MXGS = '80'
    GROUND = '92'
    SCIENCE_DATA = '02'
    INITIAL_TGF_TM ='a0'
    CONTINU_TGF_TM ='a1'
    INITIAL_BACKGR_TM ='a2'
    CONTINU_BACKGR_TM ='a3'
    INITIAL_PULSE_TM ='a4'
    CONTINU_PULSE_TM ='a5'
    INITIAL_SAMP_TM ='a6'
    CONTINU_SAMP_TM ='a7'
    INITIAL_AUR_TM ='a8'
    CONTINU_AUR_TM ='a9'

    regex = re.compile(MXGS+GROUND+SCIENCE_DATA+'a[01]....000c')



    datalist=[]

    masterdata=[]
    total=0
    import textwrap

    with open (path+fname) as f:    
        for line in f:
            if re.match(regex,line):
                datalist.append(line)
                a=datalist[i]
                dl=textwrap.wrap(a, 4)
                
                from bitstring import Bits

                f = open('file'+str(i).zfill(3)+'.ccsds', 'wb')
                Bits('0x'+line).tofile(f)
                f.close()


                total =  total + hex2int (a[8:12])
                if ( i > 0) :
                # delete first 14 bytes/28 hex (Continuation Header)
                    dl=dl[28:]
                i=i+1
                masterdata.extend(dl)

    samp={}


    x=iter(masterdata)
    samp['header']=x.next()+x.next()

    print (samp)

    samp['packet_length']                   = hex2int (x.next())
    samp['observation_id']                  = hex2int (x.next())
    samp['continuation_id']                 = hex2int (x.next())
    samp['last_packet']                     = hex2int (x.next())
    samp['utc_year ']                       = hex2int(x.next())
    samp['utc_msec']                        = hex2int(x.next())
    samp['utc_seconds']                     = hex2int (x.next()+x.next())
    samp['tcp_count_dhpu']                  = hex2int (x.next()+x.next())
    samp['tcp_count_dpu']                   = hex2int (x.next()+x.next())
    samp['dpu_count']                       = hex2int (x.next()+x.next())
    samp['dpu_count_prereset']              = hex2int (x.next()+x.next())
    samp['dau_bgo_1_int_tmon_chan1']        = hex2int (x.next())
    samp['dau_bgo_1_int_tmon_chan2']        = hex2int (x.next())
    samp['dau_bgo_1_int_tmon_chan3']        = hex2int (x.next())
    samp['dau_bgo_1_int_tmon_chan4']        = hex2int (x.next())
    samp['dau_bgo_2_int_tmon_chan1']        = hex2int (x.next())
    samp['dau_bgo_2_int_tmon_chan2']        = hex2int (x.next())
    samp['dau_bgo_2_int_tmon_chan3']        = hex2int (x.next())
    samp['dau_bgo_2_int_tmon_chan4']        = hex2int (x.next())
    samp['dau_bgo_3_int_tmon_chan1']        = hex2int (x.next())
    samp['dau_bgo_3_int_tmon_chan2']        = hex2int (x.next())
    samp['dau_bgo_3_int_tmon_chan3']        = hex2int (x.next())
    samp['dau_bgo_3_int_tmon_chan4']        = hex2int (x.next())
    samp['dau_bgo_4_int_tmon_chan1']        = hex2int (x.next())
    samp['dau_bgo_4_int_tmon_chan2']        = hex2int (x.next())
    samp['dau_bgo_4_int_tmon_chan3']        = hex2int (x.next())
    samp['dau_bgo_4_int_tmon_chan4']        = hex2int (x.next())
    samp ['led_short_win_lr_pulse_height']  = hex2int (x.next()) 
    samp ['led_short_win_upr_pulse_height'] = hex2int (x.next()) 
    samp ['led_long_win_lr_pulse_height']   = hex2int (x.next()) 
    samp ['led_long_win_upr_pulse_height']  = hex2int (x.next()) 
    samp ['hed_short_win_lr_pulse_height']  = hex2int (x.next()) 
    samp ['hed_short_win_upr_pulse_height'] = hex2int (x.next()) 
    samp ['hed_long_win_lr_pulse_height']   = hex2int (x.next()) 
    samp ['hed_long_win_upr_pulse_height']  = hex2int (x.next()) 


    shortlongwin=x.next()
    samp ['led_short_win_anticoin_time'] = hex2int (shortlongwin[0:2]) 
    samp ['led_long_win_anticoin_time']  = hex2int (shortlongwin[2:4]) 
    shortlongwin=x.next()
    samp ['hed_short_win_anticoin_time'] = hex2int (shortlongwin[0:2]) 
    samp ['hed_long_win_anticoin_time']  = hex2int (shortlongwin[2:4])

    bits=x.next()

    from bitstring import BitArray

    a = BitArray('0x'+bits)
    samp ['led_short_win_flag1'] = a[0]
    samp ['led_short_win_flag2'] = a[1]
    samp ['led_short_win_flag3'] = a[2]
    samp ['led_long_win_flag']   = a[3]
    samp ['hed_short_win_flag1'] = a[4]
    samp ['hed_short_win_flag2'] = a[5]
    samp ['hed_short_win_flag3'] = a[6]
    samp ['hed_long_win_flag']   = a[7]

    print bits

    samp ['trig_mmia_enabled'] = a[8]
    samp ['trig_mmia_recd'] = a[9]


    samp ['led_short_win_dur_1'] =  hex2int (x.next()) 
    samp ['led_short_win_dur_2'] = hex2int (x.next()) 
    samp ['led_short_win_dur_3'] = hex2int (x.next()) 
    samp ['led_long_win_dur']    = hex2int (x.next()) 
    samp ['hed_short_win_dur_1'] = hex2int (x.next()) 
    samp ['hed_short_win_dur_2'] = hex2int (x.next()) 
    samp ['hed_short_win_dur_3'] = hex2int (x.next()) 
    samp ['hed_long_win_dur']    = hex2int (x.next()) 




    scrh=x.next()

    samp ['led_short_win_thresh_1'] = hex2int (scrh[0:2])
    samp ['led_short_win_thresh_2'] = hex2int (scrh[2:4]) 

    scrh1=x.next()
    scrh2=x.next()
    scrh3=x.next()

    samp ['led_short_win_thresh_3'] = hex2int (scrh1[0:2]) 
    samp ['led_long_win_thresh']    = hex2int (scrh1[2:4]+scrh2[0:2]) 
    samp ['hed_short_win_thresh_1'] = hex2int (scrh2[2:4]) 
    samp ['hed_short_win_thresh_2'] = hex2int (scrh3[0:2]) 
    samp ['hed_short_win_thresh_3'] = hex2int (scrh3[2:4]) 
    samp ['hed_long_win_thresh']    = hex2int (x.next()) 



    samp ['mxgs_trig_count'] = hex2int (x.next()+x.next()+x.next()+x.next() )
    samp ['mxgs_trig_tcp_count'] = hex2int (x.next()+x.next()) 
    samp ['mxgs_trig_dpu_count'] = hex2int (x.next()+x.next()) 
    samp ['num_counts'] = hex2int (x.next()+x.next())




    import itertools
    det_ev=list(itertools.islice(x,  None  ))   
    arr=[i+j+k+l for i,j,k,l in zip(det_ev[::4], det_ev[1::4] , det_ev[2::4], det_ev[3::4])]

    cdfname="./tgfobservation_"+samp['header']+str(samp['observation_id']).zfill(4)+".cdf"


    mytgfobs2=MXGSTGFObservation(
        observation_id                  =samp['observation_id'],
        utc_year                        =samp['utc_year '] ,
        utc_seconds                     =samp['utc_seconds'] ,
        utc_msec                        =samp['utc_msec'] ,
        tcp_count_dhpu                  =samp['tcp_count_dhpu'], 
        tcp_count_dpu                   =samp['tcp_count_dpu'] , 
        dpu_count                   = samp ['dpu_count'],
        dpu_count_prereset          = samp ['dpu_count_prereset'],
        dau_bgo_1_int_tmon_chan1    = samp ['dau_bgo_1_int_tmon_chan1'],
        dau_bgo_1_int_tmon_chan2    = samp ['dau_bgo_1_int_tmon_chan2'],
        dau_bgo_1_int_tmon_chan3    = samp ['dau_bgo_1_int_tmon_chan3'],
        dau_bgo_1_int_tmon_chan4    = samp ['dau_bgo_1_int_tmon_chan4'],
        dau_bgo_2_int_tmon_chan1    = samp ['dau_bgo_2_int_tmon_chan1'],
        dau_bgo_2_int_tmon_chan2    = samp ['dau_bgo_2_int_tmon_chan2'],
        dau_bgo_2_int_tmon_chan3    = samp ['dau_bgo_2_int_tmon_chan3'],
        dau_bgo_2_int_tmon_chan4    = samp ['dau_bgo_2_int_tmon_chan4'],
        dau_bgo_3_int_tmon_chan1    = samp ['dau_bgo_3_int_tmon_chan1'],
        dau_bgo_3_int_tmon_chan2    = samp ['dau_bgo_3_int_tmon_chan2'],
        dau_bgo_3_int_tmon_chan3    = samp ['dau_bgo_3_int_tmon_chan3'],
        dau_bgo_3_int_tmon_chan4    = samp ['dau_bgo_3_int_tmon_chan4'],
        dau_bgo_4_int_tmon_chan1    = samp ['dau_bgo_4_int_tmon_chan1'],
        dau_bgo_4_int_tmon_chan2    = samp ['dau_bgo_4_int_tmon_chan2'],
        dau_bgo_4_int_tmon_chan3    = samp ['dau_bgo_4_int_tmon_chan3'],
        dau_bgo_4_int_tmon_chan4    = samp ['dau_bgo_4_int_tmon_chan4'],
        led_short_win_lr_pulse_height   = samp ['led_short_win_lr_pulse_height'],
        led_short_win_upr_pulse_height  = samp ['led_short_win_upr_pulse_height'],
        led_long_win_lr_pulse_height    = samp ['led_long_win_lr_pulse_height'],
        led_long_win_upr_pulse_height   = samp ['led_long_win_upr_pulse_height'],
        hed_short_win_lr_pulse_height   = samp ['hed_short_win_lr_pulse_height'],
        hed_short_win_upr_pulse_height  = samp ['hed_short_win_upr_pulse_height'],
        hed_long_win_lr_pulse_height    = samp ['hed_long_win_lr_pulse_height'],
        hed_long_win_upr_pulse_height   = samp ['hed_long_win_upr_pulse_height'],
        led_short_win_anticoin_time     = samp ['led_short_win_anticoin_time'],
        led_long_win_anticoin_time      = samp ['led_long_win_anticoin_time'],
        hed_short_win_anticoin_time     = samp ['hed_short_win_anticoin_time'],
        hed_long_win_anticoin_time      = samp ['hed_long_win_anticoin_time'],
        led_short_win_flag1             = samp ['led_short_win_flag1'],
        led_short_win_flag2             = samp ['led_short_win_flag2'],
        led_short_win_flag3             = samp ['led_short_win_flag3'],
        led_long_win_flag               = samp ['led_long_win_flag'],
        hed_short_win_flag1             = samp ['hed_short_win_flag1'],
        hed_short_win_flag2             = samp ['hed_short_win_flag2'],
        hed_short_win_flag3             = samp ['hed_short_win_flag3'],
        hed_long_win_flag               = samp ['hed_long_win_flag'],
        trig_mmia_enabled               = samp ['trig_mmia_enabled'],
        trig_mmia_recd                  = samp ['trig_mmia_recd'],
        led_short_win_dur_1       = samp ['led_short_win_dur_1'],
        led_short_win_dur_2       = samp ['led_short_win_dur_2'],
        led_short_win_dur_3       = samp ['led_short_win_dur_3'],
        led_long_win_dur          = samp ['led_long_win_dur'],
        hed_short_win_dur_1       = samp ['hed_short_win_dur_1'],
        hed_short_win_dur_2       = samp ['hed_short_win_dur_2'],
        hed_short_win_dur_3       = samp ['hed_short_win_dur_3'],
        hed_long_win_dur          = samp ['hed_long_win_dur'],
        led_short_win_thresh_1    = samp ['led_short_win_thresh_1'],
        led_short_win_thresh_2    = samp ['led_short_win_thresh_2'],
        led_short_win_thresh_3    = samp ['led_short_win_thresh_3'],
        led_long_win_thresh       = samp ['led_long_win_thresh'],
        hed_short_win_thresh_1    = samp ['hed_short_win_thresh_1'],
        hed_short_win_thresh_2    = samp ['hed_short_win_thresh_2'],
        hed_short_win_thresh_3    = samp ['hed_short_win_thresh_3'],
        hed_long_win_thresh       = samp ['hed_long_win_thresh'],
        mxgs_trig_count           = samp ['mxgs_trig_count'],
        mxgs_trig_tcp_count       = samp ['mxgs_trig_tcp_count'],
        mxgs_trig_dpu_count       = samp ['mxgs_trig_dpu_count'],
        num_counts                = samp ['num_counts'],
        detector_counts           = cdfname,
        )
    mytgfobs2.save()

    hexstr="".join(arr)
    from bitstring import Bits

    f = open('newfile.bin', 'wb')
    Bits('0x'+hexstr).tofile(f)
    f.close()


    bad_keys=['mybadkey']
    for key in sorted(samp.iterkeys()):
        if key not in bad_keys:
            value=samp[key]
            print (key, value)
            
    print "Added to database"
    import spacepy
    from spacepy import pycdf
    import os
    try:
        os.remove(cdfname)
    except OSError:
        pass
    pycdf.lib.set_backward(False)
    cdf = pycdf.CDF(cdfname, '')
    cdf['detector_counts'] = arr
    bad_keys=['mybadkey', 'continuation_id', 'last_packet']
    for key in sorted(samp.iterkeys()):
        if key not in bad_keys:
            cdf.attrs[key]=samp[key]
    cdf.attrs['Author'] = 'Gareth Murphy <gmurphy@space.dtu.dk>'
    cdf.attrs['CreateDate'] = datetime.datetime.now()
    #cdf['MMIAdata'].attrs['units'] = 'V'

    print (cdf)
    print (cdf.attrs)

    cdf.close()

In [43]:



fname='PFM_20160513_0912_raw.txt'
MXGSTGFObservation_to_db(fname)

{'header': '809202a0'}
0180
('continuation_id', 0)
('dau_bgo_1_int_tmon_chan1', 1127)
('dau_bgo_1_int_tmon_chan2', 1112)
('dau_bgo_1_int_tmon_chan3', 1112)
('dau_bgo_1_int_tmon_chan4', 3121)
('dau_bgo_2_int_tmon_chan1', 1124)
('dau_bgo_2_int_tmon_chan2', 1109)
('dau_bgo_2_int_tmon_chan3', 1107)
('dau_bgo_2_int_tmon_chan4', 3116)
('dau_bgo_3_int_tmon_chan1', 1134)
('dau_bgo_3_int_tmon_chan2', 1119)
('dau_bgo_3_int_tmon_chan3', 1117)
('dau_bgo_3_int_tmon_chan4', 3134)
('dau_bgo_4_int_tmon_chan1', 1124)
('dau_bgo_4_int_tmon_chan2', 1107)
('dau_bgo_4_int_tmon_chan3', 1107)
('dau_bgo_4_int_tmon_chan4', 3117)
('dpu_count', 644953)
('dpu_count_prereset', 1000019)
('header', '809202a0')
('hed_long_win_anticoin_time', 4)
('hed_long_win_dur', 25000)
('hed_long_win_flag', True)
('hed_long_win_lr_pulse_height', 16)
('hed_long_win_thresh', 24)
('hed_long_win_upr_pulse_height', 1023)
('hed_short_win_anticoin_time', 4)
('hed_short_win_dur_1', 300)
('hed_short_win_dur_2', 1000)
('hed_short_win_dur_3',

In [3]:
fname='PFM_20160513_0912_raw.txt'

path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/raw/'


#fname='PFM_20160513_0912_raw.txt'
i=0

import re


MXGS = '80'
GROUND = '92'
SCIENCE_DATA = '02'
INITIAL_TGF_TM ='a0'
CONTINU_TGF_TM ='a1'
INITIAL_BACKGR_TM ='a2'
CONTINU_BACKGR_TM ='a3'
INITIAL_PULSE_TM ='a4'
CONTINU_PULSE_TM ='a5'
INITIAL_SAMP_TM ='a6'
CONTINU_SAMP_TM ='a7'
INITIAL_AUR_TM ='a8'
CONTINU_AUR_TM ='a9'

regex = re.compile(MXGS+GROUND+SCIENCE_DATA+'a[01]....000c')



datalist=[]

masterdata=[]
total=0
import textwrap

with open (path+fname) as f:    
    for line in f:
        if re.match(regex,line):
            datalist.append(line)
            a=datalist[i]
            dl=textwrap.wrap(a, 4)

            from bitstring import Bits

            f = open('file'+str(i).zfill(3)+'.ccsds', 'wb')
            Bits('0x'+line).tofile(f)
            f.close()


            total =  total + hex2int (a[8:12])
            if ( i > 0) :
            # delete first 14 bytes/28 hex (Continuation Header)
                dl=dl[28:]
            i=i+1
            masterdata.extend(dl)
            
#x=MXGSTGFObservation()
#x.hexstring_to_db(masterdata)

In [6]:
x=MXGSTGFObservation()
x.hexstring_to_db(masterdata)
x.save()


import pprint
stuff = x.__dict__
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(stuff)


0180
{   '_state': <django.db.models.base.ModelState object at 0x10684b6d0>,
    'continuation_id': 0,
    'dau_bgo_1_int_tmon_chan1': 1127,
    'dau_bgo_1_int_tmon_chan2': 1112,
    'dau_bgo_1_int_tmon_chan3': 1112,
    'dau_bgo_1_int_tmon_chan4': 3121,
    'dau_bgo_2_int_tmon_chan1': 1124,
    'dau_bgo_2_int_tmon_chan2': 1109,
    'dau_bgo_2_int_tmon_chan3': 1107,
    'dau_bgo_2_int_tmon_chan4': 3116,
    'dau_bgo_3_int_tmon_chan1': 1134,
    'dau_bgo_3_int_tmon_chan2': 1119,
    'dau_bgo_3_int_tmon_chan3': 1117,
    'dau_bgo_3_int_tmon_chan4': 3134,
    'dau_bgo_4_int_tmon_chan1': 1124,
    'dau_bgo_4_int_tmon_chan2': 1107,
    'dau_bgo_4_int_tmon_chan3': 1107,
    'dau_bgo_4_int_tmon_chan4': 3117,
    'detector_counts': <FieldFile: None>,
    'dpu_count': 644953,
    'dpu_count_prereset': 1000019,
    'header': '809202a0',
    'hed_long_win_anticoin_time': 4,
    'hed_long_win_dur': 25000,
    'hed_long_win_flag': True,
    'hed_long_win_lr_pulse_height': 16,
    'hed_long_win_thre